# Creating functions
- read in base9 dataset
    - create data table of features 
    - creates random forest ml model (creating or using existing model?)
        - still need to remove the certain features (16th, 84th, mean, median, median, binary)
        - ADD 84-med(upp) & med-16 (low)
        - need to export to use later

- Add to 1st
    - read in model
    - read in NEW dataset 
    - apply model to NEW dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.io import ascii
import os
import arviz as az
from diptest import diptest
from scipy import stats

# Creating data table of features

In [2]:
def create_stats (directory, column = 0, files = 1e5): #function that will calculate all the features
    # Function to simulate MCMC samples around each mean age
    def simulate_samples(mean_age,std_dev, num_samples=10000):
        # Simulate normal distribution around the mean age
        return np.random.normal(loc=mean_age, scale=std_dev, size=num_samples)  # Adjust scale as needed

    # function to calcualate effective sample size for each star's mean age
    def calculate_ess_for_mean_ages(mean_age,std_dev_age):
        ess_values = []
        for m, s in zip(mean_age, std_dev_age):
            # Simulate MCMC samples for each mean age
            samples = simulate_samples(m, s)
            samples_reshaped = samples[np.newaxis, :]  # Reshape for ArviZ
            ess = az.ess(samples_reshaped)  # Calculate ESS
            ess_values.append(ess)
        return ess_values
    
    Median = []
    Mean = []
    Percent16 = []
    Percent84 = []
    Width=[]
    Source_id=[]
    SnR = []
    Stdev = []
    dip_val=[]
    dip_p=[]
    ks_val = []
    ks_p = []
    upper = []
    lower = []
    #creating empty arrays for each feature

    file_count = 0

    for filename in os.listdir(directory):
        if file_count >= files: 
            break 
        
        if filename.endswith(".res"):
            file_path = os.path.join(directory, filename)

            try:

                data = np.genfromtxt(file_path, skip_header=1, usecols= column)
        
        
                Age_16th = np.percentile(data, 16)
                Age_84th = np.percentile(data, 84)
                Age_med = np.median(data)
                Age_mean = np.mean(data)
                Age_wid = Age_84th-Age_16th
                Age_std = np.std(data)
                Age_snr = Age_mean / Age_std if Age_std != 0 else np.nan #inputting nan wherever dividing by zero
                Upper_bound = Age_84th - Age_med
                Lower_bound = Age_med - Age_16th

                #calculating all the features 
                

                Percent16.append(Age_16th)
                Percent84.append(Age_84th)
                Median.append(Age_med)
                Width.append(Age_wid)
                Stdev.append(Age_std)
                Mean.append(Age_mean)
                SnR.append(Age_snr)
                upper.append(Upper_bound)
                lower.append(Lower_bound)
                #appending empty arrays with calculated values
        
                source_id = filename.replace('.res','').replace('NGC_2682_','')
                Source_id.append(source_id)
                #appending the source id array with sourceid value from file name

                dip_value, p_value = diptest(data) #calculating dip value and p value
                dip_val.append(dip_value)
                dip_p.append(p_value)
                #diptest tests for unimodality .. whether a data set has a single peak or multiple
                # p value indicates whether the data is likely from a unimodial distribution (~ >.05 likely )

                normal_dist = np.random.normal(Age_mean, Age_std, len(data)) #creating an example data set with the same mean age and stdev as dataset
                ks_statistic, ks_p_value = stats.kstest(data, normal_dist)
                ks_val.append(ks_statistic)
                ks_p.append(ks_p_value)
                #ks test is comparing our dataset to the example of what a normal distribution would look like with our dataset's mean age and stdev
                #p value indicates the probablity of observing the measured difference between two distributions 

                file_count += 1

            except Exception as e:
                print(f"Error processing file {filename}: {e}")



    ESS = calculate_ess_for_mean_ages(Mean, Stdev) #adding column in for calculated ESS

    statistic = pd.DataFrame({ 
        #creating the datatable using all the calculated features
        'source_id' : Source_id,
        'Width': Width,
        'Upper_bound': Upper_bound,
        'Lower_bound': Lower_bound,
        'Stdev': Stdev,
        'SnR': SnR,
        'Dip_p': dip_p,
        'Dip_value': dip_val,
        'KS_value': ks_val,
        'KS_p': ks_p,
        'ESS': ESS



    })
    
    statistic = statistic.dropna(subset=['SnR']) #removing rows with NaN in snr column

    return statistic #output of the function will be the datatable

In [ ]:
return

In [3]:
directory = "/Users/justycewatson/Desktop/CIERA_files/NGC2682/jw_output"
#computer directory the files will come from

ngc2682_statistic = create_stats(directory)
# creating the data table using 'create_stats' function with the directory files
 
ngc2682_statistic

,source_id,Width,Upper_bound,Lower_bound,Stdev,SnR,Dip_p,Dip_value,KS_value,KS_p,ESS
0,605002016872204416,1.361236,0.557799,0.967022,0.658567,14.214572,0.000000,0.017193,0.216787,2.597879e-257,10220.351982
1,604942879467199360,1.352529,0.557799,0.967022,0.610061,15.058561,0.000000,0.019429,0.162798,3.747758e-141,10451.098943
2,604969031523465728,0.387865,0.557799,0.967022,0.263017,36.229304,0.000000,0.012712,0.149719,2.815325e-122,9677.138554
3,604921679508385664,1.676418,0.557799,0.967022,0.744099,12.044454,0.000001,0.008560,0.141053,1.188060e-107,9966.953569
4,604968958508607360,1.262185,0.557799,0.967022,0.600071,15.298958,0.000000,0.014429,0.135851,1.920291e-97,9843.157629
...,...,...,...,...,...,...,...,...,...,...,...
1423,604994045412975744,0.770949,0.557799,0.967022,0.473367,19.784625,0.000000,0.009089,0.169551,2.112362e-154,9963.197519
1424,604921924322164608,0.167193,0.557799,0.967022,0.091203,105.613775,0.000000,0.012850,0.152245,2.022412e-124,9809.822352
1425,604970062315630336,1.360707,0.557799,0.967022,0.639646,14.251756,0.000000,0.013919,0.146888,6.335192e-114,9081.034763
1426,598962292125778560,1.532101,0.557799,0.967022,0.686447,13.055923,0.000000,0.020750,0.125021,3.188440e-83,9993.445686


# Creating ML model

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

##sklearn is data analysis library

##train test split - splits arrays or matrices into random train and test subsets
##random forest classifier -  meta estimator that fits a number of decision tree classifiers on various
                            #sub-samples of the dataset and uses averaging to improve the predictive 
                            #accuracy and control over-fitting 

##accuracy score - computes subset accuracy
##select kbest - select features according to the highest scores
##f_classif - computes ANOVA (statistical test) F-value for sample
##standard scaler- standardize features by removing the mean and scaling to unit variance

In [5]:
##reading in dataframe containing known sampling quality to train model on

df1 = pd.read_csv('/Users/justycewatson/Desktop/CIERA_files/NGC2682/CSV_Files/NGC2682_Age_Stats.csv',sep=',')
sampling_df = df1[df1['Single Sampling'].isna() == False]

- change ngc2682 to broad name (model naming, replacing for source id)

In [6]:
def create_model(ngc2682_statistic, sampling_df):
  
    #making sure source_id in both dataframes are the same datatype
    ngc2682_statistic['source_id'] = ngc2682_statistic['source_id'].astype(str)
    sampling_df['source_id'] = sampling_df['source_id'].astype(str)

    #merging dataframes based on source ids to match ids to  known sampling labels
    merged_df = ngc2682_statistic.merge(
        sampling_df[['source_id', 'Single Sampling']],
        on='source_id',
        how='left'
    )
    
    merged_df = merged_df.dropna(subset=['Single Sampling']) #dropping any rows with NaN values

    X = merged_df.drop(columns=['source_id', 'Single Sampling']) #dropping sourceid and sampling columns
    y = merged_df['Single Sampling'] #target variable .. we want the model to predict these labels


    # Splitting the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    ##random state - sets a seed in random number generator, ensures the splits generated are reproducible
    ##test size - proportion of dataset to include in test split ~ 30% of data is in test split
    ##allows to train a model on training set and test its accuracy on testing set

    # Standardize the features
    scaler = StandardScaler()
    ##remove mean and scale to unit variance
    ##performs best if all features are on the same scale, (0-1, or whatever scale)


    X_train = scaler.fit_transform(X_train)

    X_test_save = X_test
    X_test = scaler.transform(X_test)
    ##fit - computes mean and stdevs to be used for scaling
    ##transform - perform scaling using mean and stdev from .fit

    #Can use selector if wanting to run with different k 
    # # Feature selection to identify the most relevant features
    # selector = SelectKBest(f_classif, k='all')  # Adjust 'k' to select fewer features if needed
    # X_train_selected = selector.fit_transform(X_train, y_train)
    # X_test_selected = selector.transform(X_test)
    # ##select best features according to k highest scores ?
    # ##k is the amount of features



    # Train a Random Forest Classifier
    clf = RandomForestClassifier(random_state=42)
    clf.fit(X_train, y_train)
    ## training a model by feeding it a dataset and rfc learns patterns and relationships within data to make predictions on new unseen data

    return X, y, clf, X_test, X_train, y_test

def make_preds(X, clf, y_test = None, X_columns = None):
    # Make predictions
    y_pred = clf.predict(X)
    ##making predictions from rfc 
    #predicting what label would be


    #Print evaluation metrics
    #Giving option to print important features
    if y_test is not None:
        print("Accuracy:", accuracy_score(y_test, y_pred))
        print(classification_report(y_test, y_pred))
    ##computing subset accuracy, label predictions in a sample must exactly corresponding labels in y
    ##classification report - build a text report showing main classification metrics

        # Feature importance
        feature_importances = clf.feature_importances_
        important_features = pd.Series(feature_importances, index=X_columns).sort_values(ascending=False)
        ##measuring how much a feature contributes to a model's prediction 



        print("Feature Importance Ranking:")
        print(important_features)

    return y_pred
    

- repolace xtest with 6819 np array
- no y_test
- xcolumns is 6819array columns

- another cell making plot comparing with elizabeths labels like the histogram/bar chart

In [7]:
X, y, clf, X_test, X_train, y_test = create_model(ngc2682_statistic, sampling_df)
y_pred = make_preds(X_test, clf, y_test = y_test, X_columns = X.columns)

Accuracy: 0.969626168224299
              precision    recall  f1-score   support

         Bad       0.99      0.98      0.98       359
        Good       0.89      0.93      0.91        69

    accuracy                           0.97       428
   macro avg       0.94      0.95      0.94       428
weighted avg       0.97      0.97      0.97       428

Feature Importance Ranking:
Width          0.349902
SnR            0.226814
Stdev          0.159254
KS_value       0.152734
Dip_value      0.070850
ESS            0.028525
Dip_p          0.009100
KS_p           0.002820
Upper_bound    0.000000
Lower_bound    0.000000
dtype: float64


/var/folders/69/cbqkcsfx3z1_8353d2qb0fj40000gn/T/ipykernel_58430/3555142146.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampling_df['source_id'] = sampling_df['source_id'].astype(str)


In [8]:
X

,Width,Upper_bound,Lower_bound,Stdev,SnR,Dip_p,Dip_value,KS_value,KS_p,ESS
0,1.361236,0.557799,0.967022,0.658567,14.214572,0.000000,0.017193,0.216787,2.597879e-257,10220.351982
1,1.352529,0.557799,0.967022,0.610061,15.058561,0.000000,0.019429,0.162798,3.747758e-141,10451.098943
2,0.387865,0.557799,0.967022,0.263017,36.229304,0.000000,0.012712,0.149719,2.815325e-122,9677.138554
3,1.676418,0.557799,0.967022,0.744099,12.044454,0.000001,0.008560,0.141053,1.188060e-107,9966.953569
4,1.262185,0.557799,0.967022,0.600071,15.298958,0.000000,0.014429,0.135851,1.920291e-97,9843.157629
...,...,...,...,...,...,...,...,...,...,...
1422,0.770949,0.557799,0.967022,0.473367,19.784625,0.000000,0.009089,0.169551,2.112362e-154,9963.197519
1423,0.167193,0.557799,0.967022,0.091203,105.613775,0.000000,0.012850,0.152245,2.022412e-124,9809.822352
1424,1.360707,0.557799,0.967022,0.639646,14.251756,0.000000,0.013919,0.146888,6.335192e-114,9081.034763
1425,1.532101,0.557799,0.967022,0.686447,13.055923,0.000000,0.020750,0.125021,3.188440e-83,9993.445686


In [9]:
ngc2682_statistic

,source_id,Width,Upper_bound,Lower_bound,Stdev,SnR,Dip_p,Dip_value,KS_value,KS_p,ESS
0,605002016872204416,1.361236,0.557799,0.967022,0.658567,14.214572,0.000000,0.017193,0.216787,2.597879e-257,10220.351982
1,604942879467199360,1.352529,0.557799,0.967022,0.610061,15.058561,0.000000,0.019429,0.162798,3.747758e-141,10451.098943
2,604969031523465728,0.387865,0.557799,0.967022,0.263017,36.229304,0.000000,0.012712,0.149719,2.815325e-122,9677.138554
3,604921679508385664,1.676418,0.557799,0.967022,0.744099,12.044454,0.000001,0.008560,0.141053,1.188060e-107,9966.953569
4,604968958508607360,1.262185,0.557799,0.967022,0.600071,15.298958,0.000000,0.014429,0.135851,1.920291e-97,9843.157629
...,...,...,...,...,...,...,...,...,...,...,...
1423,604994045412975744,0.770949,0.557799,0.967022,0.473367,19.784625,0.000000,0.009089,0.169551,2.112362e-154,9963.197519
1424,604921924322164608,0.167193,0.557799,0.967022,0.091203,105.613775,0.000000,0.012850,0.152245,2.022412e-124,9809.822352
1425,604970062315630336,1.360707,0.557799,0.967022,0.639646,14.251756,0.000000,0.013919,0.146888,6.335192e-114,9081.034763
1426,598962292125778560,1.532101,0.557799,0.967022,0.686447,13.055923,0.000000,0.020750,0.125021,3.188440e-83,9993.445686


# Saving Model


- save model fn3

import pickle 
filename = 'my_model.pkl'  (any name .pkl)
pickle.dump(model, open(filename, 'wb'))   (saving)
loaded_model = pickle.load(open(filename, 'rb'))   (loading back in)

In [10]:
import pickle

def save_model(ngc2682_model, filename = 'my_model.pkl'):
    pickle.dump(ngc2682_model, open(filename, 'wb'))

#creating a function that will save the model to a pickle file to be used later

# Applying new dataset to saved model

- fn4 sending model to function, sending new dataset to it (6819) and returning the ml labels

Reusing first function to create stats df to make sure everything is exactly the same

- how to reuse first function and get the course ids correct? Or should I just load in file ? 

In [11]:
X_test

array([[ 0.3295017 ,  0.        ,  0.        , ...,  0.01676116,
        -0.03201477,  0.76606463],
       [ 0.36216461,  0.        ,  0.        , ..., -0.24559673,
        -0.03201477,  0.86177007],
       [-1.62262396,  0.        ,  0.        , ..., -0.02012735,
        -0.03201477,  0.55496402],
       ...,
       [ 0.58676434,  0.        ,  0.        , ...,  1.4185245 ,
        -0.03201477,  0.22040608],
       [-1.31238031,  0.        ,  0.        , ...,  0.4024572 ,
        -0.03201477,  1.15465014],
       [ 0.8261269 ,  0.        ,  0.        , ..., -0.48192827,
        -0.03201477,  0.49276955]])

In [12]:
directory = '/Users/justycewatson/Desktop/CIERA_files/NGC6819/ngc6819_single_resfiles' 

ngc6819_statistic = create_stats(directory)
# creating the data table using 'create_stats' function with the directory files
 
ngc6819_statistic

#need to send model same columns in xtest 
#convert columns from ngc6819 to np array -- send to make preds function

,source_id,Width,Upper_bound,Lower_bound,Stdev,SnR,Dip_p,Dip_value,KS_value,KS_p,ESS
0,gaia_2076279137552154112_sin2,1.611105,0.281932,0.241271,0.709164,12.619723,0.0,0.015125,0.163108,2.433374e-146,10230.814740
1,gaia_2076298933067389056_sin2,0.851659,0.281932,0.241271,0.492672,18.154950,0.0,0.075300,0.253279,0.000000e+00,9811.851657
2,gaia_2076584355102288128_sin2,1.568087,0.281932,0.241271,0.706918,12.674155,0.0,0.015795,0.167611,4.184633e-152,10227.789678
3,gaia_2076488525792609792_sin2,0.384405,0.281932,0.241271,0.494811,18.364221,0.0,0.052020,0.282041,0.000000e+00,10174.095869
4,gaia_2076581851147039232_sin2,1.684027,0.281932,0.241271,0.748072,11.787783,0.0,0.021270,0.181134,9.422417e-178,9989.953156
...,...,...,...,...,...,...,...,...,...,...,...
1683,gaia_2076393834663325568_sin2,1.420736,0.281932,0.241271,0.685752,12.768550,0.0,0.009732,0.165418,1.575146e-150,9866.895149
1684,gaia_2076380400005524224_sin2,1.370467,0.281932,0.241271,0.686048,12.883365,0.0,0.021400,0.133522,1.823723e-96,10103.163570
1685,gaia_2076299895139969536_sin2,0.444381,0.281932,0.241271,0.175729,54.419198,0.0,0.085369,0.166908,4.819513e-152,9600.976567
1686,gaia_2076393662864610688_sin2,1.519873,0.281932,0.241271,0.696125,12.398271,0.0,0.016644,0.160876,2.362612e-142,9671.862079


In [13]:
ngc6819 = ngc6819_statistic.to_numpy()
ngc6819_statistic

,source_id,Width,Upper_bound,Lower_bound,Stdev,SnR,Dip_p,Dip_value,KS_value,KS_p,ESS
0,gaia_2076279137552154112_sin2,1.611105,0.281932,0.241271,0.709164,12.619723,0.0,0.015125,0.163108,2.433374e-146,10230.814740
1,gaia_2076298933067389056_sin2,0.851659,0.281932,0.241271,0.492672,18.154950,0.0,0.075300,0.253279,0.000000e+00,9811.851657
2,gaia_2076584355102288128_sin2,1.568087,0.281932,0.241271,0.706918,12.674155,0.0,0.015795,0.167611,4.184633e-152,10227.789678
3,gaia_2076488525792609792_sin2,0.384405,0.281932,0.241271,0.494811,18.364221,0.0,0.052020,0.282041,0.000000e+00,10174.095869
4,gaia_2076581851147039232_sin2,1.684027,0.281932,0.241271,0.748072,11.787783,0.0,0.021270,0.181134,9.422417e-178,9989.953156
...,...,...,...,...,...,...,...,...,...,...,...
1683,gaia_2076393834663325568_sin2,1.420736,0.281932,0.241271,0.685752,12.768550,0.0,0.009732,0.165418,1.575146e-150,9866.895149
1684,gaia_2076380400005524224_sin2,1.370467,0.281932,0.241271,0.686048,12.883365,0.0,0.021400,0.133522,1.823723e-96,10103.163570
1685,gaia_2076299895139969536_sin2,0.444381,0.281932,0.241271,0.175729,54.419198,0.0,0.085369,0.166908,4.819513e-152,9600.976567
1686,gaia_2076393662864610688_sin2,1.519873,0.281932,0.241271,0.696125,12.398271,0.0,0.016644,0.160876,2.362612e-142,9671.862079


- what am i sending to makepreds for 6819? 

Questions

-  should i include the df with sampling labels as an input or just read it in as a file in the function? 
- do i need to retun all those things in the function ? 
- should i include source id for 6819 ? How will we know which stars are which ? 